In [1]:
import os

def check_subfolders_in_images(directory_path):
    """
    Checks the number of subfolders inside 'Images_001' for each main folder.

    Args:
    - directory_path (str): The main directory containing subfolders.

    Prints:
    - Number of subfolders inside 'Images_001' for each folder.
    """
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        folders = [f for f in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, f))]
        
        for folder in folders:
            image_001_path = os.path.join(directory_path, folder, "Images_001")
            if os.path.exists(image_001_path) and os.path.isdir(image_001_path):
                subfolders = [sf for sf in os.listdir(image_001_path) if os.path.isdir(os.path.join(image_001_path, sf))]
                print(f"Subfolders inside '{image_001_path}': {len(subfolders)}")
            else:
                print(f"No 'Images_001' folder found inside '{folder}'.")
    else:
        print(f"Directory '{directory_path}' does not exist or is not a directory.")

# Your directory path
directory_path = r"E:\2 Below Lake Waco Deep 28_8 Concentrated"
check_subfolders_in_images(directory_path)


Subfolders inside 'E:\2 Below Lake Waco Deep 28_8 Concentrated\2_38 Below Lake Waco Deep 28_8 Gal 9-17-21 Image1a\Images_001': 671
Subfolders inside 'E:\2 Below Lake Waco Deep 28_8 Concentrated\2_38 Below Lake Waco Deep 28_8 Gal 9-17-21 Image2a\Images_001': 387
Subfolders inside 'E:\2 Below Lake Waco Deep 28_8 Concentrated\2_38 Below Lake Waco Deep 28_8 Gal 9-17-21 Image3a\Images_001': 352
Subfolders inside 'E:\2 Below Lake Waco Deep 28_8 Concentrated\2_38 Below Lake Waco Deep 28_8 Gal 9-17-21 Image4a\Images_001': 277
Subfolders inside 'E:\2 Below Lake Waco Deep 28_8 Concentrated\2_38 Below Lake Waco Deep 28_8 Gal 9-17-21 Image5a\Images_001': 22
Subfolders inside 'E:\2 Below Lake Waco Deep 28_8 Concentrated\2_38 Below Lake Waco Deep 28_8 Gal 9-17-21 Image6a\Images_001': 789


In [ ]:
import os
import numpy as np
from tqdm import tqdm

try:
    from tensorflow.keras.preprocessing.image import load_img, img_to_array
except ImportError:
    print("TensorFlow is not installed. You can install it by running: pip install tensorflow")
    raise

def load_images_from_subfolders(directory_path):
    """
    Loads the first five images from each subfolder inside 'Images_001' for each main folder.
    Resizes the images to (28, 28, 3) for deep learning models.
    Pads with the last image if fewer than 5 images are available.
    Stores all subfolders' images in a single array and prints its shape.

    Args:
    - directory_path (str): The main directory containing subfolders.

    Prints:
    - Shape of the array containing resized images for all subfolders.
    """
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        folders = [f for f in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, f))]
        
        for folder in tqdm(folders, desc="Processing main folders"):
            image_001_path = os.path.join(directory_path, folder, "Images_001")
            if os.path.exists(image_001_path) and os.path.isdir(image_001_path):
                subfolders = [sf for sf in os.listdir(image_001_path) if os.path.isdir(os.path.join(image_001_path, sf))]
                
                # Array to store all images for the current main folder
                all_images = []

                for subfolder in tqdm(subfolders, desc=f"Processing subfolders in '{folder}'", leave=False):
                    subfolder_path = os.path.join(image_001_path, subfolder)
                    image_files = [f for f in os.listdir(subfolder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                    
                    images = []
                    for image_file in image_files[:5]:
                        image_path = os.path.join(subfolder_path, image_file)
                        img = load_img(image_path, target_size=(28, 28))
                        img_array = img_to_array(img) / 255.0  # Normalize pixel values
                        images.append(img_array)

                    # Pad with the last image if there are fewer than 5
                    while len(images) < 5:
                        images.append(images[-1])

                    all_images.append(np.array(images))  # Add (5, 28, 28, 3) to the batch array

                # Convert to a numpy array of shape (num_subfolders, 5, 28, 28, 3)
                all_images_array = np.array(all_images)
                print(f"\nSubfolders inside '{image_001_path}': shape of images: {all_images_array.shape}")
            else:
                print(f"\nNo 'Images_001' folder found inside '{folder}'.")
    else:
        print(f"\nDirectory '{directory_path}' does not exist or is not a directory.")

# Your directory path
directory_path = r"E:\2 Below Lake Waco Deep 28_8 Concentrated"
load_images_from_subfolders(directory_path)


In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tqdm import tqdm

# Custom PatchEncoder layer
class PatchEncoder(tf.keras.layers.Layer):
    def __init__(self, num_patches, projection_dim, **kwargs):
        super(PatchEncoder, self).__init__(**kwargs)
        self.num_patches = num_patches
        self.projection_dim = projection_dim
        self.projection = tf.keras.Sequential(
            [
                tf.keras.layers.Conv2D(2, (3, 3), strides=(1, 1), activation='relu'),
                tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling2D((2, 2))),
                tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()),
                tf.keras.layers.Dense(projection_dim),
            ]
        )
        self.position_embedding = tf.keras.layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def get_config(self):
        config = super().get_config()
        config.update({
            "num_patches": self.num_patches,
            "projection_dim": self.projection_dim,
        })
        return config

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

# Load model
model_path = r"E:\five_frame_model.h5"
custom_objects = {'PatchEncoder': PatchEncoder}

try:
    model = load_model(model_path, custom_objects=custom_objects)
    print("\nModel loaded successfully!")
except Exception as e:
    print(f"\nError loading model: {e}")
    raise

def load_images_from_subfolders(directory_path):
    """
    Loads the first five images from each subfolder inside 'Images_001' for each main folder.
    Resizes the images to (28, 28, 3) for deep learning models.
    Pads with the last image if fewer than 5 images are available.
    """
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        folders = [f for f in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, f))]
        
        for folder in tqdm(folders, desc="Processing main folders"):
            image_001_path = os.path.join(directory_path, folder, "Images_001")
            if os.path.exists(image_001_path) and os.path.isdir(image_001_path):
                subfolders = [sf for sf in os.listdir(image_001_path) if os.path.isdir(os.path.join(image_001_path, sf))]
                all_images = []

                for subfolder in tqdm(subfolders, desc=f"Processing subfolders in '{folder}'", leave=False):
                    subfolder_path = os.path.join(image_001_path, subfolder)
                    image_files = [f for f in os.listdir(subfolder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

                    images = []
                    for image_file in image_files[:5]:
                        image_path = os.path.join(subfolder_path, image_file)
                        img = load_img(image_path, target_size=(28, 28))
                        img_array = img_to_array(img) / 255.0  # Normalize pixel values
                        images.append(img_array)

                    # Pad with the last image if there are fewer than 5
                    while len(images) < 5:
                        images.append(images[-1])

                    all_images.append(np.array(images))  # Add (5, 28, 28, 3) to the batch array

                # Convert to a numpy array of shape (num_subfolders, 5, 28, 28, 3)
                all_images_array = np.array(all_images)
                print(f"\nSubfolders inside '{image_001_path}': shape of images: {all_images_array.shape}")

                # Make individual predictions for the current folder
                make_predictions(model, all_images_array, folder)
            else:
                print(f"\nNo 'Images_001' folder found inside '{folder}'.")
    else:
        print(f"\nDirectory '{directory_path}' does not exist or is not a directory.")

def make_predictions(model, data, folder_name):
    """
    Makes predictions on the given data and prints the count of each class for a specific folder.
    """
    print(f"\nMaking predictions for folder '{folder_name}'...")
    predictions = model.predict(data, batch_size=32, verbose=1)
    predicted_classes = np.argmax(predictions, axis=1)
    unique_classes, counts = np.unique(predicted_classes, return_counts=True)
    print(f"\nPredicted Class Distribution for '{folder_name}':")
    for cls, count in zip(unique_classes, counts):
        print(f"Class {cls}: {count}")

# Load images and make individual predictions
directory_path = r"E:\2 Below Lake Waco Deep 28_8 Concentrated"
#load_images_from_subfolders(directory_path)



Model loaded successfully!


In [4]:
# Load images and make individual predictions
directory_path = r"G:\datasets\2.Veliger_Data\67 Red Bud Sample 1 Kayak Net_2\67 Red Bud Sample 1 Kayak Net"
load_images_from_subfolders(directory_path)


Processing main folders:   0%|                                                                   | 0/6 [00:00<?, ?it/s]
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':   0%|    | 0/879 [00:00<?, ?it/s]
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':   0%| | 1/879 [00:00<02:14,  6.51
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':   0%| | 2/879 [00:00<02:39,  5.49
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':   0%| | 3/879 [00:00<02:22,  6.15
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':   0%| | 4/879 [00:00<02:04,  7.04
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':   1%| | 5/879 [00:00<02:17,  6.36
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':   1%| | 7/879 [00:01<02:04,  6.98
Processing subfolders in '67_41 Red Bud 

Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':   8%| | 68/879 [00:14<01:59,  6.7
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':   8%| | 69/879 [00:14<02:07,  6.3
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':   8%| | 70/879 [00:14<02:14,  6.0
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':   8%| | 71/879 [00:14<02:06,  6.4
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':   8%| | 72/879 [00:14<02:09,  6.2
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':   8%| | 73/879 [00:15<02:22,  5.6
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':   8%| | 74/879 [00:15<02:26,  5.4
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':   9%| | 75/879 [00:15<02:38,  5.0
Processing subfolders in '67_41 Red Bud 

Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':  15%|▏| 136/879 [00:27<02:02,  6.
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':  16%|▏| 137/879 [00:27<02:03,  6.
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':  16%|▏| 138/879 [00:27<02:08,  5.
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':  16%|▏| 139/879 [00:27<02:16,  5.
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':  16%|▏| 140/879 [00:28<02:20,  5.
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':  16%|▏| 141/879 [00:28<02:12,  5.
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':  16%|▏| 142/879 [00:28<02:09,  5.
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':  16%|▏| 143/879 [00:28<02:26,  5.
Processing subfolders in '67_41 Red Bud 

Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':  23%|▏| 202/879 [00:42<01:52,  6.
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':  23%|▏| 203/879 [00:42<01:46,  6.
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':  23%|▏| 205/879 [00:43<01:34,  7.
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':  23%|▏| 206/879 [00:43<01:38,  6.
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':  24%|▏| 207/879 [00:43<01:36,  7.
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':  24%|▏| 208/879 [00:43<01:39,  6.
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':  24%|▏| 209/879 [00:43<01:31,  7.
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':  24%|▏| 210/879 [00:43<01:30,  7.
Processing subfolders in '67_41 Red Bud 

Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':  32%|▎| 282/879 [00:53<00:58, 10.
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':  32%|▎| 284/879 [00:53<00:57, 10.
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':  33%|▎| 286/879 [00:54<01:04,  9.
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':  33%|▎| 288/879 [00:54<01:02,  9.
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':  33%|▎| 290/879 [00:54<00:56, 10.
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':  33%|▎| 292/879 [00:54<00:56, 10.
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':  33%|▎| 294/879 [00:54<00:54, 10.
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image1':  34%|▎| 296/879 [00:54<00:59,  9.
Processing subfolders in '67_41 Red Bud 

...

Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image6':  93%|▉| 1420/1529 [02:15<00:08, 1
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image6':  93%|▉| 1422/1529 [02:15<00:07, 1
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image6':  93%|▉| 1424/1529 [02:15<00:06, 1
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image6':  93%|▉| 1426/1529 [02:15<00:06, 1
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image6':  93%|▉| 1428/1529 [02:15<00:06, 1
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image6':  94%|▉| 1430/1529 [02:15<00:06, 1
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image6':  94%|▉| 1432/1529 [02:15<00:06, 1
Processing subfolders in '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image6':  94%|▉| 1434/1529 [02:16<00:06, 1
Processing subfolders in '67_41 Red Bud 


Subfolders inside 'G:\datasets\2.Veliger_Data\67 Red Bud Sample 1 Kayak Net_2\67 Red Bud Sample 1 Kayak Net\67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image6\Images_001': shape of images: (1529, 5, 28, 28, 3)

Making predictions for folder '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image6'...
48/48 [==============================] - 1s 25ms/step


Processing main folders: 100%|██████████████████████████████████████████████████████████| 6/6 [10:18<00:00, 116.74s/it]


Predicted Class Distribution for '67_41 Red Bud Sample 1 Kayak Net 5-27-2022 12_50AM Image6':
Class 0: 1515
Class 1: 14


Processing main folders: 100%|██████████████████████████████████████████████████████████| 6/6 [10:18<00:00, 103.06s/it]
